# AgentCore Long-Term Memory Extractor

This notebook demonstrates how to extract and display the current values stored in AWS Bedrock AgentCore long-term memory.

## Prerequisites
- AWS credentials configured
- `bedrock-agentcore` package installed
- AgentCore Memory permissions

In [ ]:
# Install required packages if needed
# !pip install bedrock-agentcore boto3

In [1]:
import logging
from typing import List, Dict, Any
from bedrock_agentcore.memory.client import MemoryClient
from bedrock_agentcore.memory.session import MemorySessionManager

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration
REGION = "us-west-2"  # Change to your region
MEMORY_ID = None      # Set your memory ID here, or leave None to discover

print("✅ Imports completed")

✅ Imports completed


## Step 1: Discover Available Memories

In [2]:
# Initialize memory client
memory_client = MemoryClient(region_name=REGION)

# List all available memories
memories = memory_client.list_memories()

print(f"📋 Found {len(memories)} memory resources:")
print("-" * 60)

for i, memory in enumerate(memories, 1):
    memory_id = memory.get('memoryId') or memory.get('id')
    name = memory.get('name', 'Unnamed')
    status = memory.get('status', 'Unknown')
    
    print(f"{i}. Memory ID: {memory_id}")
    print(f"   Name: {name}")
    print(f"   Status: {status}")
    print()

# # Use first memory if MEMORY_ID not set
# if not MEMORY_ID and memories:
#     MEMORY_ID = memories[0].get('memoryId') or memories[0].get('id')
    
#     print(f"🎯 Using memory: {MEMORY_ID}")
# elif not memories:
#     print("❌ No memories found. Create a memory resource first.")


MEMORY_ID ="TravelAgent_STM_20251012114311-E9nNWYBpoW"

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:bedrock_agentcore.memory.client:Initialized MemoryClient for control plane: us-west-2, data plane: us-west-2


📋 Found 18 memory resources:
------------------------------------------------------------
1. Memory ID: AgentCore_AgentPath_Test_09_01-krREPC7SAw
   Name: Unnamed
   Status: ACTIVE

2. Memory ID: CulinaryAssistant-std1w98kF2
   Name: Unnamed
   Status: ACTIVE

3. Memory ID: CustomerSupportMemory-atl8uPHrQ1
   Name: Unnamed
   Status: ACTIVE

4. Memory ID: FitnessCoach-3fnwf68RvO
   Name: Unnamed
   Status: ACTIVE

5. Memory ID: MathLanggraphAgent-P5b365Bmsf
   Name: Unnamed
   Status: ACTIVE

6. Memory ID: PersonalAgentMemory-I7mOd16Oow
   Name: Unnamed
   Status: ACTIVE

7. Memory ID: RuntimeMemoryAgent_4d14614f-owT0UbA0v6
   Name: Unnamed
   Status: ACTIVE

8. Memory ID: RuntimeMemoryAgent_5bc25ada-xaGOx0BtdC
   Name: Unnamed
   Status: ACTIVE

9. Memory ID: SecureAgentMemory_9fef60-c90gpC9Ue6
   Name: Unnamed
   Status: ACTIVE

10. Memory ID: SupportAgent-mi0U8uHsH9
   Name: Unnamed
   Status: ACTIVE

11. Memory ID: TravelAgent_LTM-czzpY471Ke
   Name: Unnamed
   Status: ACTIVE

12. 

## Step 2: Get Memory Configuration

In [3]:
if MEMORY_ID:
    # Get detailed memory information
    memory_details = memory_client.gmcp_client.get_memory(memoryId=MEMORY_ID)
    memory = memory_details["memory"]
    
    print(f"📄 MEMORY DETAILS:")
    print("-" * 40)
    print(f"ID: {memory.get('id', 'N/A')}")
    print(f"Name: {memory.get('name', 'N/A')}")
    print(f"Status: {memory.get('status', 'N/A')}")
    print(f"Description: {memory.get('description', 'N/A')}")
    print(f"Created: {memory.get('createdAt', 'N/A')}")
    print(f"Event Expiry: {memory.get('eventExpiryDuration', 'N/A')} days")
    
    # Display strategies
    strategies = memory.get('strategies', [])
    print(f"\n🧠 MEMORY STRATEGIES ({len(strategies)} configured):")
    print("-" * 50)
    
    all_namespaces = set()
    
    for i, strategy in enumerate(strategies, 1):
        strategy_id = strategy.get('strategyId', 'N/A')
        strategy_name = strategy.get('name', 'Unnamed')
        strategy_type = strategy.get('type', 'Unknown')
        strategy_status = strategy.get('status', 'Unknown')
        namespaces = strategy.get('namespaces', [])
        
        print(f"{i}. {strategy_name} ({strategy_type})")
        print(f"   ID: {strategy_id}")
        print(f"   Status: {strategy_status}")
        print(f"   Namespaces: {namespaces}")
        print()
        
        # Collect namespaces
        all_namespaces.update(namespaces)
    
    print(f"📂 Total unique namespaces: {len(all_namespaces)}")
    for ns in sorted(all_namespaces):
        print(f"   - {ns}")
else:
    print("❌ No memory ID available")

📄 MEMORY DETAILS:
----------------------------------------
ID: TravelAgent_STM_20251012114311-E9nNWYBpoW
Name: TravelAgent_STM_20251012114311
Status: ACTIVE
Description: Travel Agent STM
Created: 2025-10-12 11:43:13.759000-07:00
Event Expiry: 7 days

🧠 MEMORY STRATEGIES (3 configured):
--------------------------------------------------
1. memory_preference_bgh1l (USER_PREFERENCE)
   ID: memory_preference_bgh1l-UT98mFGiMi
   Status: ACTIVE
   Namespaces: ['/strategies/{memoryStrategyId}/actors/{actorId}']

2. memory_semantic_bgh1l (SEMANTIC)
   ID: memory_semantic_bgh1l-fx8JFbAyji
   Status: ACTIVE
   Namespaces: ['/strategies/{memoryStrategyId}/actors/{actorId}']

3. summary_builtin_bgh1l (SUMMARIZATION)
   ID: summary_builtin_bgh1l-Z0BuDjBDAx
   Status: ACTIVE
   Namespaces: ['/strategies/{memoryStrategyId}/actors/{actorId}/sessions/{sessionId}']

📂 Total unique namespaces: 2
   - /strategies/{memoryStrategyId}/actors/{actorId}
   - /strategies/{memoryStrategyId}/actors/{actorId}/sess

## Step 3: Extract Memory Records

In [4]:
if MEMORY_ID:
    # Initialize session manager
    session_manager = MemorySessionManager(memory_id=MEMORY_ID, region_name=REGION)
    
    print(f"📦 EXTRACTING MEMORY RECORDS:")
    print("=" * 60)
    
    total_records = 0
    
    # Extract records from each namespace
    for namespace in sorted(all_namespaces):
        print(f"\n🔍 Searching namespace: {namespace}")
        print("-" * 40)
        
        try:
            # List records in this namespace
            records = session_manager.list_long_term_memory_records(
                namespace_prefix=namespace,
                max_results=20
            )
            
            if records:
                print(f"✅ Found {len(records)} records:")
                
                for j, record in enumerate(records, 1):
                    record_id = record.get('memoryRecordId', 'N/A')
                    created = record.get('createdAt', 'N/A')
                    updated = record.get('updatedAt', 'N/A')
                    
                    # Extract content
                    content = record.get('content', {})
                    text_content = content.get('text', 'No content available')
                    
                    # Display truncated content
                    display_text = text_content[:200] + "..." if len(text_content) > 200 else text_content
                    
                    print(f"\n   📄 Record {j}:")
                    print(f"      ID: {record_id}")
                    print(f"      Created: {created}")
                    print(f"      Updated: {updated}")
                    print(f"      Content: {display_text}")
                
                total_records += len(records)
            else:
                print("   No records found")
                
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    print(f"\n{'='*60}")
    print(f"TOTAL RECORDS EXTRACTED: {total_records}")
    print(f"{'='*60}")
else:
    print("❌ No memory ID available for extraction")

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:bedrock_agentcore.memory.session:  -> Listing all long-term records in namespace '/strategies/{memoryStrategyId}/actors/{actorId}'...


📦 EXTRACTING MEMORY RECORDS:

🔍 Searching namespace: /strategies/{memoryStrategyId}/actors/{actorId}
----------------------------------------


ERROR:bedrock_agentcore.memory.session:     ❌ Error listing long-term records: An error occurred (ValidationException) when calling the ListMemoryRecords operation: 1 validation error detected: Value at 'namespace' failed to satisfy constraint: Member must satisfy regular expression pattern: [a-zA-Z0-9/*][a-zA-Z0-9-_/*]*(?::[a-zA-Z0-9-_/*]+)*[a-zA-Z0-9-_/*]*
INFO:bedrock_agentcore.memory.session:  -> Listing all long-term records in namespace '/strategies/{memoryStrategyId}/actors/{actorId}/sessions/{sessionId}'...
ERROR:bedrock_agentcore.memory.session:     ❌ Error listing long-term records: An error occurred (ValidationException) when calling the ListMemoryRecords operation: 1 validation error detected: Value at 'namespace' failed to satisfy constraint: Member must satisfy regular expression pattern: [a-zA-Z0-9/*][a-zA-Z0-9-_/*]*(?::[a-zA-Z0-9-_/*]+)*[a-zA-Z0-9-_/*]*


   ❌ Error: An error occurred (ValidationException) when calling the ListMemoryRecords operation: 1 validation error detected: Value at 'namespace' failed to satisfy constraint: Member must satisfy regular expression pattern: [a-zA-Z0-9/*][a-zA-Z0-9-_/*]*(?::[a-zA-Z0-9-_/*]+)*[a-zA-Z0-9-_/*]*

🔍 Searching namespace: /strategies/{memoryStrategyId}/actors/{actorId}/sessions/{sessionId}
----------------------------------------
   ❌ Error: An error occurred (ValidationException) when calling the ListMemoryRecords operation: 1 validation error detected: Value at 'namespace' failed to satisfy constraint: Member must satisfy regular expression pattern: [a-zA-Z0-9/*][a-zA-Z0-9-_/*]*(?::[a-zA-Z0-9-_/*]+)*[a-zA-Z0-9-_/*]*

TOTAL RECORDS EXTRACTED: 0


## Step 4: Search Memory Records (if no records found above)

In [ ]:
if MEMORY_ID and total_records == 0:
    print(f"🔍 NO RECORDS FOUND - TRYING SEMANTIC SEARCH:")
    print("=" * 60)
    
    # Try common search queries
    search_queries = ["user", "customer", "conversation", "interaction", "support", "order", "help"]
    
    search_results_found = 0
    
    for namespace in all_namespaces:
        print(f"\n🎯 Searching in namespace: {namespace}")
        
        for query in search_queries:
            try:
                search_results = session_manager.search_long_term_memories(
                    query=query,
                    namespace_prefix=namespace,
                    top_k=3
                )
                
                if search_results:
                    print(f"\n   ✅ Query '{query}' found {len(search_results)} results:")
                    
                    for k, record in enumerate(search_results, 1):
                        content = record.get('content', {}).get('text', 'No content')
                        score = record.get('relevanceScore', 0)
                        
                        # Truncate content
                        display_text = content[:150] + "..." if len(content) > 150 else content
                        
                        print(f"      {k}. Score: {score:.3f}")
                        print(f"         Content: {display_text}")
                    
                    search_results_found += len(search_results)
                    break  # Found results, move to next namespace
                    
            except Exception as e:
                continue
    
    if search_results_found == 0:
        print("\n💡 TROUBLESHOOTING TIPS:")
        print("-" * 30)
        print("• Memory might be empty or still processing")
        print("• Check if events have been stored using create_event()")
        print("• Verify memory strategies are in ACTIVE status")
        print("• Wait for memory extraction to complete (2-3 minutes after storing events)")
        print("• Check namespace patterns match your stored data")
    else:
        print(f"\n✅ Found {search_results_found} records through semantic search")
elif MEMORY_ID:
    print(f"✅ Memory extraction completed successfully!")
else:
    print("❌ No memory available for search")

## Step 5: Summary and Next Steps

In [ ]:
if MEMORY_ID:
    print(f"\n📊 EXTRACTION SUMMARY:")
    print("=" * 40)
    print(f"Memory ID: {MEMORY_ID}")
    print(f"Memory Name: {memory.get('name', 'N/A')}")
    print(f"Strategies: {len(strategies)}")
    print(f"Namespaces: {len(all_namespaces)}")
    print(f"Records Found: {total_records}")
    
    if total_records > 0:
        print("\n✅ SUCCESS: Memory contents extracted successfully!")
        print("\n📝 NEXT STEPS:")
        print("• Analyze the extracted content for insights")
        print("• Use search_long_term_memories() for specific queries")
        print("• Implement memory retrieval in your agents")
    else:
        print("\n⚠️  NO RECORDS FOUND")
        print("\n📝 NEXT STEPS:")
        print("• Store some events using create_event() or add_turns()")
        print("• Wait 2-3 minutes for memory extraction to process")
        print("• Verify your memory strategies are ACTIVE")
        print("• Check namespace patterns match your data")
else:
    print("❌ No memory found. Create a memory resource first using MemoryClient.create_memory()")